In [7]:
import torch
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import numpy as np
import time
import os

In [8]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]


ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [9]:
print(source_points)

[[61.84826691 40.32822479  4.5       ]
 [99.10094831 54.69940709  2.        ]
 [99.89962676 24.72759871  2.        ]
 [23.54499552 57.03946784  2.        ]
 [25.09781584 22.62636785  2.        ]]


In [10]:
model = PINN([200,200,200,200])
model.set_location(source_points,[1,x_max,y_max,z_max],source_values=[0,10,0,0,0],sigma = .1)
tfinal = 1.

In [11]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=4e-3)
from torch.optim.lr_scheduler import ExponentialLR

scheduler = ExponentialLR(optimizer, gamma=0.995)  # Decay LR by 5% every epoch

In [12]:
n=5000
sn = 3000
best_loss = np.inf
print_freq = 100
for epoch in range(5000):
    start_time = time.time()
    if epoch % 20 == 0:
        source_collocation_points = model.source_points(2000,.1)
        ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max*1.5, torch.rand(sn,1)*y_max*1.5, torch.rand(sn,1)*z_max*1.5], dim=1)

        collocation_points = torch.cat([torch.rand(n,1)*tfinal*1.5, torch.rand(n,1)*x_max*1.5, torch.rand(n,1)*y_max*1.5, torch.rand(n,1)*z_max*1.5], dim=1)
        collocation_points = torch.empty(0,4)
        # ic_col = torch.empty(0,4)
        ic_col.requires_grad_()
        collocation_points = torch.cat([collocation_points,ic_col,source_collocation_points])
        uv = torch.ones(len(collocation_points),2)*.5
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.loss_function(collocation_points,uv) # PDE residual loss
    # loss_2,pde_2 = model.loss_function(source_collocation_points, uv_points, source_term = source_values) # source term PDE residual loss 
    # loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_collocation_points]),torch.concat([collocation_points,source_collocation_points]))

    loss = loss_1 
    # loss = loss_2*100
    # loss = loss_1

    # # print loss at first epoch
    # if epoch == 0:
    #     print('epoch: %d, loss: %1.3e, pde_res: %1.3e, source_loss: %1.3e' % (epoch, loss.item(), loss_1.item(), loss_2.item()))

    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    for p in model.net.parameters():
        grad_norm += p.grad.data.norm(2).item()**2
    grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    # scheduler.step()

    if epoch % print_freq == 0:

        # print epoch and loss using %1.3e format
        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), epoch_time))



epoch: 0, loss: 1.220e+02, grad_norm: 3.196e+02, pde_res: 1.220e+02, time: 8.695e-01
epoch: 100, loss: 4.170e+01, grad_norm: 2.031e+02, pde_res: 4.170e+01, time: 2.631e-02
epoch: 200, loss: 6.042e+01, grad_norm: 1.018e+02, pde_res: 6.042e+01, time: 2.528e-02
epoch: 300, loss: 3.416e+01, grad_norm: 2.032e+02, pde_res: 3.416e+01, time: 2.573e-02
epoch: 400, loss: 3.148e+01, grad_norm: 8.010e+01, pde_res: 3.148e+01, time: 2.485e-02
epoch: 500, loss: 1.836e+01, grad_norm: 9.846e+01, pde_res: 1.836e+01, time: 2.313e-02
epoch: 600, loss: 5.146e+01, grad_norm: 6.621e+01, pde_res: 5.146e+01, time: 2.430e-02
epoch: 700, loss: 3.664e+01, grad_norm: 1.449e+02, pde_res: 3.664e+01, time: 2.484e-02


KeyboardInterrupt: 

In [13]:
source_collocation_points

tensor([[ 0.6738, 98.9835, 54.8135,  2.0938],
        [ 0.2572, 99.0988, 54.7350,  2.0975],
        [ 0.5798, 99.0592, 54.9738,  1.9071],
        ...,
        [ 0.1691, 99.2060, 54.6626,  1.9112],
        [ 0.7276, 99.0460, 54.7771,  2.0300],
        [ 0.1757, 99.0297, 54.8280,  1.9655]])

In [14]:
# model = torch.load('best_mod.pth')

In [15]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = 2
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, x_max, n)
y_grid = np.linspace(0, y_max, n)
z_grid = np.linspace(0, 2, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 20)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

'\n# Plot the concentration over time\nfor t in time_steps:\n    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input\n    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape\n    \n    plt.figure()\n    plt.contourf(X, Y, concentration, levels=50, cmap=\'viridis\', vmin=0, vmax=10)  # Plot concentration as a contour plot\n    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), \'ro\', label=\'Source Location\')  # Plot the source location\n    plt.colorbar(label=\'Concentration\')\n    # fix colorbar from 0 to 1 \n    # plt.clim(0, 10.0)  # Set colorbar limits\n    plt.title(f"Gas Concentration at t = {t:.2f}")\n    plt.xlabel(\'x\')\n    plt.ylabel(\'y\')\n    # fix colorbar \n    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image\n'

In [16]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=False).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[1,0], model.source_locs[1,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.title(f"t = {t:.2f}")

    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual.gif', images)  # Save the images as a GIF



C:\Users\andyh\AppData\Local\Temp\ipykernel_24840\1417289318.py:23: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [17]:
model.source_mixture
source_term = torch.tensor(np.exp(model.source_mixture.score_samples(model.source_locs)))
source_term

tensor([ 0.0000, 20.0785,  0.0000,  0.0000,  0.0000], dtype=torch.float64)

In [ ]:
print(len(model.net.hidden))

5


In [22]:
print(model.source_mixture.weights_)

[10]
